### 實作BP神經網路模型 (以mnist 手寫數字辨識為例)  
- 資料讀取  
讀進 keras 的手寫數字資料集，分別為圖形(像素值的矩陣)以及資料標籤(0-9)。  
為進行訓練並代入模型，將訓練及測試集的圖形像素值調整至0-1之間，並將28*28的矩陣拉成長為784的向量。

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import random

mnist = keras.datasets.mnist

(train_images,train_labels),(test_images,test_labels) = mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

# input_mat 設定 (訓練資料矩陣)
train_size = train_images.shape[0]
test_size = test_images.shape[0]
input_mat = np.reshape(train_images, (train_size, train_images.shape[1]*train_images.shape[2]))

input_mat.shape # 確認要代入模型的資料大小

(60000, 784)

設定訓練集的標籤 y ，為代入模型將標籤改為label coding的形式。

In [9]:
# ymat 設定 (訓練資料標籤改為label coding的形式)

ymat = np.repeat(0, 10*train_size).reshape(train_size, 10)

for i in range(train_size):
    ymat[i, train_labels[i]] = 1

- BP神經網路實作

In [10]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

random.seed(0)

# 參數設定
learning_rate = 0.2

n_in = input_mat.shape[1]
n_hidden = [20, 20] # 隱藏層兩層 各有?, ?個neuron
n_out = ymat.shape[1]

# 第1層權重初始值
w = np.random.randn(n_hidden[0], n_in)

# 第2層權重初始值
w2 = np.random.randn(n_hidden[1], n_hidden[0])

# 第3層權重初始值
w3 = np.random.randn(n_out, n_hidden[1])

t = 0
output_, y = ymat[0], ymat[0]+1

while True:
    
    input_ = input_mat[t % input_mat.shape[0]]
    y = ymat[t % input_mat.shape[0]]

    ### -----正向傳播-----

    # 計算output
    hidden_1 = sigmoid(np.matmul(w, input_))
    hidden_2 = sigmoid(np.matmul(w2, hidden_1))
    output_ = sigmoid(np.matmul(w3, hidden_2))


    ### -----誤差反向傳播-----

    # 第3層權重delta計算
    delta3 = (y - output_) * output_ * (1 - output_)

    # 第2層權重delta計算
    delta2 = hidden_2 * (1 - hidden_2) * np.matmul(delta3, w3)

    # 第1層權重delta計算
    delta1 = hidden_1 * (1 - hidden_1) * np.matmul(delta2, w2)

    # 第3層權重修正
    w3_old = w3.copy()
    w3 += learning_rate * np.tensordot(delta3, hidden_2, axes = 0)

    # 第2層權重修正
    w2_old = w2.copy()
    w2 += learning_rate * np.tensordot(delta2, hidden_1, axes = 0)

    # 第1層權重修正
    w_old = w.copy()
    w += learning_rate * np.tensordot(delta1, input_, axes = 0)

    t += 1

    if t > 500000 or \
        (np.sum(abs(w3_old - w3)) + np.sum(abs(w2_old - w2)) + \
            np.sum(abs(w_old - w)) < 0.000001).all():
        print('t=',t) # 總共跑了 t 次
        break

t= 475736


- 驗證資料集  
將驗證資料代入模型，並將 output 改為標籤(0-9)的形式。

In [11]:
# 驗證資料集
test_mat = np.reshape(test_images, (test_size, test_images.shape[1]*test_images.shape[2])).T

# 將驗證資料集代入模型
test_hidden_1 = sigmoid(np.matmul(w, test_mat))
test_hidden_2 = sigmoid(np.matmul(w2, test_hidden_1))
test_output_ = sigmoid(np.matmul(w3, test_hidden_2))

In [12]:
# 將output 改為標籤的形式

output_y = np.zeros(test_size)

for i in range(test_size):
    for j in range(10):
        if test_output_.T[i, j] >= max(test_output_.T[i, ]):
            output_y[i] = j

output_y # 確認 output 已改為數字0-9的標籤

array([7., 2., 1., ..., 4., 5., 6.])

- 檢驗模型準確率

In [13]:
print('test accuracy = {}'.format(sum(test_labels == output_y)/test_size))

test accuracy = 0.9262
